In [18]:
from datetime import datetime
from datetime import timedelta
from datetime import date


class PolicyHolder:
    def __init__(self, holder_id, name, email):
        self.holder_id = holder_id
        self.name = name
        self.email = email
        self.active = True
        self.policies = []

    def suspend(self):
        self.active = False

    def activate(self):
        self.active = True

    def add_policy(self, policy):
        self.policies.append(policy)

    def __str__(self):
        return f"{self.name} ({'Active' if self.active else 'Suspended'})"

class PolicyProduct:
    def __init__(self, product_id, name, premium, coverage):
        self.product_id = product_id
        self.name = name
        self.premium = premium
        self.coverage = coverage
        self.active = True  

    def suspend(self):
        self.active = False

    def activate(self):
        self.active = True
    def __str__(self):
        status = "Active" if self.active else "Suspended"
        return f"{self.name} - Premium: ${self.premium}, Coverage: {self.coverage} ({status})"

class Payment:
    def __init__(self, payment_id, holder_id, amount, due_date, date_paid=None):
        self.payment_id = payment_id
        self.holder_id = holder_id
        self.amount = amount
        self.date = date or datetime.now()
        self.due_date = due_date
        self.date_paid = date_paid
        self.status = "Paid" if date_paid else "Pending"

    def mark_paid(self, date_paid=None):
        self.date_paid = date_paid or datetime.now()
        self.status = "Paid"

    def is_overdue(self):
        return self.status == "Pending" and datetime.now().date() > self.due_date

    def __str__(self):
        return f"Payment {self.payment_id} - ${self.amount} - {self.status} (Due: {self.due_date})"


class PolicyManagementSystem:
    def __init__(self):
        self.policyholders = {}
        self.products = {}
        self.payments = []

    def add_policyholder(self, holder_id, name, email):
        self.policyholders[holder_id] = PolicyHolder(holder_id, name, email)

    def suspend_policyholder(self, holder_id):
        if holder_id in self.policyholders:
            self.policyholders[holder_id].suspend()

    def register_product(self, product_id, name, premium, coverage):
        self.products[product_id] = PolicyProduct(product_id, name, premium, coverage)

    def assign_policy(self, holder_id, product_id):
        if holder_id in self.policyholders and product_id in self.products:
            self.policyholders[holder_id].add_policy(self.products[product_id])

    def suspend_product(self, product_id):
        if product_id in self.products:
            self.products[product_id].suspend()
        else:
            print("Product not found.")

    def reactivate_product(self, product_id):
        if product_id in self.products:
            self.products[product_id].activate()
        else:
            print("Product not found.")

    def show_all_assigned_policies(self):
        for holder_id, holder in self.policyholders.items():
            print(f"\nPolicyholder: {holder.name} (ID: {holder_id})")
            if holder.policies:
                for policy in holder.policies:
                    print(f"  - {policy}")
            else:
                print("  No policies assigned.")

    def record_payment(self, payment_id, holder_id, amount):
        payment = Payment(payment_id, holder_id, amount)
        self.payments.append(payment)

    def schedule_payment(self, payment_id, holder_id, amount, due_date):
        payment = Payment(payment_id, holder_id, amount, due_date)
        self.payments.append(payment)

    def process_payment(self, payment_id, date_paid=None):
        for payment in self.payments:
            if payment.payment_id == payment_id:
                payment.mark_paid(date_paid)
                return
        print("Payment not found.")

    def send_payment_reminders(self):
        print("\n--- Payment Reminders ---")
        for payment in self.payments:
            if payment.status == "Pending":
                days_left = (payment.due_date - datetime.now().date()).days
                if days_left <= 3:
                    print(f"Reminder: Payment {payment.payment_id} for Holder {payment.holder_id} is due in {days_left} day(s).")

    def apply_penalties(self, penalty_amount):
        print("\n--- Applying Penalties ---")
        for payment in self.payments:
            if payment.is_overdue():
                print(f"Penalty applied to Holder {payment.holder_id} for overdue Payment {payment.payment_id}.")
                payment.amount += penalty_amount


    def get_policyholder_info(self, holder_id):
        return str(self.policyholders.get(holder_id, "Policyholder not found"))

    def get_all_payments(self):
        return [str(payment) for payment in self.payments]

#to manage the interactions
pms = PolicyManagementSystem()

# Add policyholders
pms.add_policyholder("PH025", "Lia", "lia@example.com")
pms.add_policyholder("PH026", "Kinta", "kinta@example.com")
pms.add_policyholder("PH029", "Reza", "reza@example.com")

# Register products
pms.register_product("P001", "Health Insurance", 1000, "Medical")
pms.register_product("P002", "Life Insurance", 2000, "Life")

# Assign policies
pms.assign_policy("PH025", "P002")
pms.assign_policy("PH026", "P001")
pms.assign_policy("PH029", "P002")

# Schedule payments
pms.schedule_payment("PAY003", "PH025", 500, date(2025, 7, 25))
pms.schedule_payment("PAY004", "PH026", 1000, date(2025, 7, 20))
pms.schedule_payment("PAY004", "PH029", 2000, date(2025, 7, 22))

# Send reminders
pms.send_payment_reminders()

# Process a payment
pms.process_payment("PAY003")

# Display info
print(pms.get_policyholder_info("PH025"))
print(pms.get_policyholder_info("PH026"))
print(pms.get_policyholder_info("PH029"))
print(pms.get_all_payments())


--- Payment Reminders ---
Reminder: Payment PAY003 for Holder PH025 is due in 1 day(s).
Reminder: Payment PAY004 for Holder PH026 is due in -4 day(s).
Reminder: Payment PAY004 for Holder PH029 is due in -2 day(s).
Lia (Active)
Kinta (Active)
Reza (Active)
['Payment PAY003 - $500 - Paid (Due: 2025-07-25)', 'Payment PAY004 - $1000 - Pending (Due: 2025-07-20)', 'Payment PAY004 - $2000 - Pending (Due: 2025-07-22)']


In [19]:
# Apply penalties for overdue payments
pms.apply_penalties(50)
print(pms.get_all_payments())


--- Applying Penalties ---
Penalty applied to Holder PH026 for overdue Payment PAY004.
Penalty applied to Holder PH029 for overdue Payment PAY004.
['Payment PAY003 - $500 - Paid (Due: 2025-07-25)', 'Payment PAY004 - $1050 - Pending (Due: 2025-07-20)', 'Payment PAY004 - $2050 - Pending (Due: 2025-07-22)']
